In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/ O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-diabetes/X_test.csv
/kaggle/input/2021-ml-diabetes/submit.csv
/kaggle/input/2021-ml-diabetes/data.csv


In [2]:
RANDOM_STATE = 42
file_dir = '/kaggle/input/2021-ml-diabetes/'

train = pd.read_csv(file_dir+'data.csv')
X_test = pd.read_csv(file_dir+'X_test.csv')
submit = pd.read_csv(file_dir+'submit.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

## 데이터 확인

In [4]:
train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
0,1,95,60,18,58,23.9,0.260,22,0
1,5,105,72,29,325,36.9,0.159,28,0
2,0,135,68,42,250,42.3,0.365,24,1
3,4,131,68,21,166,33.1,0.160,28,0
4,1,103,30,38,83,43.3,0.183,33,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               537 non-null    int64  
 1   Glucose                   537 non-null    int64  
 2   BloodPressure             537 non-null    int64  
 3   SkinThickness             537 non-null    int64  
 4   Insulin                   537 non-null    int64  
 5   BMI                       537 non-null    float64
 6   DiabetesPedigreeFunction  537 non-null    float64
 7   Age                       537 non-null    int64  
 8   Diabetes                  537 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 37.9 KB


In [6]:
y_train = train['Diabetes']
X_train = train.drop('Diabetes', axis=1)

In [7]:
y_train.value_counts()

0    349
1    188
Name: Diabetes, dtype: int64

In [8]:
X_train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,3.726257,120.849162,69.685289,20.432030,79.836127,31.975605,0.469920,33.074488
std,3.262965,32.339523,18.094374,15.490715,115.196730,7.624495,0.342087,11.685319
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,64.000000,0.000000,0.000000,26.800000,0.241000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,36.000000,32.000000,0.374000,29.000000
75%,6.000000,139.000000,80.000000,32.000000,129.000000,36.500000,0.612000,41.000000
max,17.000000,199.000000,122.000000,63.000000,846.000000,67.100000,2.420000,81.000000


## standard scaling

In [9]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)

X_train_std = std_scaler.transform(X_train)
X_test_std = std_scaler.transform(X_test)

## 하이퍼파라미터 조정없이 확인

In [10]:
def check_score_whitout_params(model, X, y, metric, 
                               test_size=None, 
                               stratify=None,
                               random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=stratify)
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print(f'train: {metric(y_train, y_train_pred)}')
    print(f'test: {metric(y_test, y_test_pred)}')

### LDA

#### standard scaling -> X

In [11]:
check_score_whitout_params(LinearDiscriminantAnalysis(),
                           X_train, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.7935323383084577
test: 0.7555555555555555


In [12]:
# cld=LinearDiscriminantAnalysis(store_covariance=True)
# cld.fit(X_train, y_train)

# y_test_pred = cld.predict(X_test)
# submit['Diabetes'] = y_test_pred
# submit.to_csv('submission_LDA_no_params.csv', index=False)

#### standard scaling -> O

In [13]:
check_score_whitout_params(LinearDiscriminantAnalysis(),
                           X_train_std, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.7935323383084577
test: 0.7555555555555555


In [14]:
# cld_std=LinearDiscriminantAnalysis(store_covariance=True)
# cld_std.fit(X_train_std, y_train)

# y_test_std_pred = cld_std.predict(X_test_std)
# submit['Diabetes'] = y_test_std_pred
# submit.to_csv('submission_LDA_std_no_params.csv', index=False)

### QDA

#### standard scaling -> X

In [15]:
check_score_whitout_params(QuadraticDiscriminantAnalysis(),
                           X_train, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.7686567164179104
test: 0.7037037037037037


In [16]:
cqd=QuadraticDiscriminantAnalysis(store_covariance=True)
cqd.fit(X_train, y_train)

y_test_pred = cqd.predict(X_test)
submit['Diabetes'] = y_test_pred
submit.to_csv('submission_QDA_no_params.csv', index=False)

#### standard scaling -> O

In [17]:
check_score_whitout_params(QuadraticDiscriminantAnalysis(),
                           X_train_std, y_train, 
                           accuracy_score, 
                           random_state=RANDOM_STATE)

train: 0.7686567164179104
test: 0.7037037037037037


In [18]:
# cqd_std = QuadraticDiscriminantAnalysis(store_covariance=True)
# cqd_std.fit(X_train_std, y_train)

# y_test_std_pred = cqd_std.predict(X_test_std)
# submit['Diabetes'] = y_test_std_pred
# submit.to_csv('submission_QDA_std_no_params.csv', index=False)